In [1]:
import json
import datetime
import pytz
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from statistics import mean
import statsmodels.api as sm
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from statistics import mean
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler  
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
import datetime,time

In [2]:
path = "tweet_data/"

files = ["tweets_#gohawks.txt", "tweets_#gopatriots.txt", \
        "tweets_#nfl.txt", "tweets_#patriots.txt", \
        "tweets_#sb49.txt", "tweets_#superbowl.txt"]
topics = ["gohawks", "gopatriots", "nfl", "patriots", "sb49", "superbowl"]
test_files =["sample0_period1.txt","sample0_period2.txt","sample0_period3.txt", \
             "sample1_period1.txt","sample1_period2.txt","sample1_period3.txt","sample2_period1.txt",\
             "sample2_period2.txt","sample2_period3.txt"]

for i, fl in enumerate(files):
    print("files[" + str(i) + "] => " + fl)

files[0] => tweets_#gohawks.txt
files[1] => tweets_#gopatriots.txt
files[2] => tweets_#nfl.txt
files[3] => tweets_#patriots.txt
files[4] => tweets_#sb49.txt
files[5] => tweets_#superbowl.txt


In [3]:

def parse_dataset(path,files):
    """
    Parse x conponent of the dataset into pandas DataFrame including columns of:
    tweets, retweets, total_followers, max_followers, mentioned, media, active, author, favourites_count, title
    with lines of hours,
    where mentioned: number of @ in tweets per hour
          media: number of url in tweets per hour
          active: a measure of active state of author
          author: number of unique authors post tweet per hour
          favourites_count: the total number of likes by this user
          title: length of this tweet's title
    Parse y of dataset as number of tweets in the next hour.
    """
    start_time = time.mktime(time.strptime("2015-02-01 08:00:00",'%Y-%m-%d %H:%M:%S'))
    end_time = time.mktime(time.strptime("2015-02-01 20:00:00",'%Y-%m-%d %H:%M:%S'))   

    start_hour_idx = ((2-1)*31+1-14)*24+8
    end_hour_idx = ((2-1)*31+1-14)*24+20
    start_minute_idx = (((2-1)*31+1-14)*24 + (8-0))*12 + 0//5    
        
    # extract raw features
    pst_tz = pytz.timezone('America/Los_Angeles')
    p = datetime.datetime.fromtimestamp(start_time, pst_tz) 
#     print("start_time=",start_time,"date=",p)
    data_raw = [[],[],[]]
    for file in files:
        for line in open(path + file, 'r', encoding="utf-8") :
            row_tmp = []
            a = json.loads(line)
            citation_date = a['citation_date']
            tweet = 1
            retweet = a['metrics']['citations']['total']
            foll = a['author']['followers']
            hr_min=1
            p = datetime.datetime.fromtimestamp(citation_date, pst_tz) 
#             print("citation_date=",citation_date,"date=",p)
            # append to row_tmp
            row_tmp.append(citation_date)        
            row_tmp.append(tweet)        
            row_tmp.append(retweet)
            row_tmp.append(foll)  
            row_tmp.append(hr_min) 
            # assign to 3 periods
            if citation_date < start_time:
                data_raw[0].append(row_tmp)
            elif citation_date < end_time:
                data_raw[1].append(row_tmp)
            else:
                data_raw[2].append(row_tmp)            
#         print("next file")
    # generate raw pandas dataframe
    pddata_raw_1 = transfer_time(data_raw[0],'hour')
    pddata_raw_1['time'] = pddata_raw_1['time'] - pddata_raw_1.loc[0,'time']
    
    pddata_raw_2 = transfer_time(data_raw[1],'minute')
#     pddata_raw_2['time'] = pddata_raw_2['time'] - start_minute_idx
    pddata_raw_2['time'] = pddata_raw_2['time'] - pddata_raw_2.loc[0,'time']

    pddata_raw_3 = transfer_time(data_raw[2],'hour')
#     pddata_raw_3['time'] = pddata_raw_3['time'] - end_hour_idx - 1    
    pddata_raw_3['time'] = pddata_raw_3['time'] - pddata_raw_3.loc[0,'time']
    #print('1',pddata_raw_1)
    
    # generate df and df_y for each time slot
    df_1 = generate_df(pddata_raw_1)  
    df_y_1 = df_1.iloc[1:,1].reset_index(drop=True)
    df_1 = df_1[:len(df_y_1)]
    #print('2',df_1)
    
    df_2 = generate_df(pddata_raw_2)
    df_y_2 = df_2.iloc[1:,1].reset_index(drop=True)
    df_2 = df_2[:len(df_y_2)]
   
    df_3 = generate_df(pddata_raw_3)
#     print(df_3)
    df_y_3 = df_3.iloc[1:,1].reset_index(drop=True)
    df_3 = df_3[:len(df_y_3)]
    
    return (df_1.iloc[:,1:],df_y_1), (df_2.iloc[:,1:],df_y_2), (df_3.iloc[:,1:],df_y_3)

In [4]:
def transfer_time(data_raw,time_type):
    
    pst_tz = pytz.timezone('America/Los_Angeles')
    
    # sort according to time
    pddata_raw = pd.DataFrame(data_raw,columns=['time','tweets','retweets','followers','hr_min'])
    pddata_raw = pddata_raw.sort_values(by = 'time')
    pddata_raw = pddata_raw.reset_index(drop=True)               

    # calculate hour index and minute index from time
    if time_type == 'hour':
        hour_accu = []
        hour_min = []
        for index, row in pddata_raw.iterrows():  
            p = datetime.datetime.fromtimestamp(row['time'], pst_tz)  
            hour_accu.append(((p.month-1)*31+p.day)*24+p.hour)  
            hour_min.append(p.hour)
        pddata_raw['time'] = hour_accu
        pddata_raw['hr_min'] = hour_min
    elif time_type == 'minute':
        minu_accu = []
        hour_min = []
        for index, row in pddata_raw.iterrows():  
            p = datetime.datetime.fromtimestamp(row['time'], pst_tz)                    
            minu_accu.append((((p.month-1)*31+p.day)*24 + (p.hour-0))*12 + p.minute//5)   
            hour_min.append(p.minute//5)
        pddata_raw['time'] = minu_accu    
        pddata_raw['hr_min'] = hour_min
    else:
        print("Invalid time type")
        
    return pddata_raw

In [5]:
def generate_df(pddata_raw):
    """
    Create a new dataframe with desired form
    """
    df = pd.DataFrame([],columns=['time unit','tweets','retweets','followers sum','followers max','hr_min'])
    
    col = pddata_raw.columns.get_loc('time')
    df['time unit'] = range(int(pddata_raw.iloc[len(pddata_raw.index)-1,col] - pddata_raw.iloc[0,col]+1))
    df['tweets'] = pddata_raw.groupby("time")['tweets'].sum()
    df['retweets'] = pddata_raw.groupby("time")['retweets'].sum()
    df['followers sum'] = pddata_raw.groupby("time")['followers'].sum()
    df['followers max'] = pddata_raw.groupby("time")["followers"].max()
    df['hr_min'] = pddata_raw.groupby("time")['hr_min'].mean()        
    # reset index of df
    df = df.fillna(0).reset_index(drop=True)
    
    return df

In [6]:
def plot_recipe(df_y, pred_y):
    """
    This function plots fitted values vs true values
    """
    plt.figure()
    area = np.pi * (4)**2/4
    plt.scatter(df_y, pred_y, s = area)
    plt.plot([df_y.min(), df_y.max()], [df_y.min(), df_y.max()], 'k--', lw = 1)
    plt.xlabel('true values')
    plt.ylabel('fitted values')
    plt.show()

In [6]:

def ols_regression(df, df_y):
    X2 = sm.add_constant(df)
    y = df_y.as_matrix()
    lm = sm.OLS(y, X2).fit()
    print(lm.summary())
    print(list(df))

In [8]:
(df,df_y),(df_2,df_y_2),(df_3,df_y_3) = parse_dataset(path,files)

In [17]:
param_grid={
'max_depth': [10, 20, 40, 60, 80, 100, 200, None],
'max_features': ['auto', 'sqrt'],
'min_samples_leaf': [1, 2, 4],
'min_samples_split': [2, 5, 10],
'n_estimators': [10,20,50,100,200, 400, 600, 800, 1000,
1200, 1400, 1600, 1800, 2000]
}
kf = KFold(n_splits=5,random_state=42,shuffle=True)

In [10]:
df.head()

,tweets,retweets,followers sum,followers max,mentioned,media,active,author,favourites_count,title
0,111,714,223405.0,41818.0,45,14,16495.717600,68,169019,107.738739
1,89,663,187317.0,19558.0,25,23,14668.183788,63,126515,114.955056
2,110,640,1657090.0,1362401.0,30,20,18070.444596,68,219997,118.100000
3,100,267,165256.0,19558.0,25,14,15166.468933,66,96787,118.460000
4,137,468,444858.0,65150.0,77,18,11300.612887,96,164014,111.883212


In [35]:
df_y_2.head()

0    1336
1    1231
2    1299
3    1230
4     321
Name: tweets, dtype: int64

# GradientBoostingRegressor 

In [20]:

avg_RMSE1_train=np.zeros((8,2,3,3,14))
avg_RMSE1_test=np.zeros((8,2,3,3,14))
for n1,max_depth in enumerate(param_grid['max_depth']):
    for n2,max_features in enumerate(param_grid['max_features']):
        for n3,min_samples_leaf in enumerate(param_grid['min_samples_leaf']):
            for n4,min_samples_split in enumerate(param_grid['min_samples_split']):
                for n5,n_estimators in enumerate(param_grid['n_estimators']):
                    MSE_train=[]
                    MSE_test=[]
                    oob_error=[]
                    total_train=0
                    total_test=0
                    for train_index, test_index in kf.split(df):
                        X_train= df.iloc[train_index]
                        y_train= df_y.iloc[train_index]
                        X_test= df.iloc[test_index]
                        y_test= df_y.iloc[test_index]
                        reg = GradientBoostingRegressor(n_estimators=n_estimators,max_depth=max_depth,min_samples_leaf=min_samples_leaf,min_samples_split=min_samples_split, max_features=max_features,random_state=42)
                        reg.fit(X_train,y_train)
                        pred_train = reg.predict(X_train)
                        pred_test = reg.predict(X_test)
                        MSE_train.append(mean_squared_error(y_train, pred_train)*len(train_index))
                        MSE_test.append(mean_squared_error(y_test, pred_test)*len(test_index))
                        total_train=total_train+len(train_index)
                        total_test=total_test+len(test_index)

                    avg_RMSE1_test[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_test)/total_test)
                    avg_RMSE1_train[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_train)/total_train)
                    print("max_depth=",max_depth,"max_features=",max_features,"min_samples_leaf=",min_samples_leaf,"min_samples_split=",min_samples_split,"n_estimators=",n_estimators)
                    print("RMSE_train=",avg_RMSE1_train[n1,n2,n3,n4,n5],"RMSE_test=",avg_RMSE1_test[n1,n2,n3,n4,n5])
                    print("")

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 10
RMSE_train= 957.771401766056 RMSE_test= 2496.124567901489

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 20
RMSE_train= 336.3596869777686 RMSE_test= 2597.278040316634

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 50
RMSE_train= 16.181756480638935 RMSE_test= 2685.933778855037

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 100
RMSE_train= 0.7705088336179384 RMSE_test= 2690.2665214243157

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 200
RMSE_train= 0.016015693871191717 RMSE_test= 2690.257330411056

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 400
RMSE_train= 0.00031450320438490906 RMSE_test= 2690.25643798525

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estima

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1600
RMSE_train= 0.00031444446534423205 RMSE_test= 2784.505715634007

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1800
RMSE_train= 0.000314444465344258 RMSE_test= 2784.505715634007

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 2000
RMSE_train= 0.000314444465344258 RMSE_test= 2784.505715634007

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 10
RMSE_train= 1362.7961363989843 RMSE_test= 2381.7481980024054

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 20
RMSE_train= 837.1370547459892 RMSE_test= 2462.5579475688646

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 50
RMSE_train= 273.0114200881412 RMSE_test= 2760.2754325226465

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 5

max_depth= 10 max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1000
RMSE_train= 0.08131105322763205 RMSE_test= 2566.2989901515402

max_depth= 10 max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1200
RMSE_train= 0.01561484574923441 RMSE_test= 2566.2981542076877

max_depth= 10 max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1400
RMSE_train= 0.0036837475374487737 RMSE_test= 2566.2978619625105

max_depth= 10 max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1600
RMSE_train= 0.0010909123660742242 RMSE_test= 2566.2978136599017

max_depth= 10 max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1800
RMSE_train= 0.0004606120593643662 RMSE_test= 2566.2978041340075

max_depth= 10 max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 2000
RMSE_train= 0.00033066482951484894 RMSE_test= 2566.297804063952

max_depth= 10 max_features= auto min_samples_leaf= 4 min

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 400
RMSE_train= 0.19758162287143421 RMSE_test= 2266.090098410152

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 600
RMSE_train= 0.008641530263265779 RMSE_test= 2266.087476257815

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 800
RMSE_train= 0.00044443915225682193 RMSE_test= 2266.0875458250093

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 1000
RMSE_train= 0.00031499952640949764 RMSE_test= 2266.0875470945844

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 1200
RMSE_train= 0.0003149995264100133 RMSE_test= 2266.0875470945844

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 1400
RMSE_train= 0.00031499952641003795 RMSE_test= 2266.0875470945844

max_depth= 10 max_features= sqrt min_samples_leaf= 

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 50
RMSE_train= 728.4971352891415 RMSE_test= 2158.264639270541

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 100
RMSE_train= 334.9472039112744 RMSE_test= 2169.5337646653625

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 200
RMSE_train= 110.77364203867525 RMSE_test= 2209.245135958578

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 400
RMSE_train= 12.938620388866092 RMSE_test= 2220.3634300110766

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 600
RMSE_train= 1.8588290127879263 RMSE_test= 2222.0787007824442

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 800
RMSE_train= 0.25143041085765794 RMSE_test= 2222.3263598752687

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_esti

...

max_depth= 200 max_features= sqrt min_samples_leaf= 4 min_samples_split= 10 n_estimators= 100
RMSE_train= 372.6526249719781 RMSE_test= 2207.777755690809

max_depth= 200 max_features= sqrt min_samples_leaf= 4 min_samples_split= 10 n_estimators= 200
RMSE_train= 132.9765519255098 RMSE_test= 2261.668459611245

max_depth= 200 max_features= sqrt min_samples_leaf= 4 min_samples_split= 10 n_estimators= 400
RMSE_train= 16.15085546958598 RMSE_test= 2279.485675280353

max_depth= 200 max_features= sqrt min_samples_leaf= 4 min_samples_split= 10 n_estimators= 600
RMSE_train= 2.362920676984255 RMSE_test= 2281.092852653215

max_depth= 200 max_features= sqrt min_samples_leaf= 4 min_samples_split= 10 n_estimators= 800
RMSE_train= 0.34370099309174656 RMSE_test= 2281.4034082343287

max_depth= 200 max_features= sqrt min_samples_leaf= 4 min_samples_split= 10 n_estimators= 1000
RMSE_train= 0.05014104495187486 RMSE_test= 2281.445464618482

max_depth= 200 max_features= sqrt min_samples_leaf= 4 min_samples_spli

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 10 n_estimators= 2000
RMSE_train= 0.00031461989619075846 RMSE_test= 2941.8121855239856

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 10
RMSE_train= 1339.7512242428907 RMSE_test= 2444.7699597673677

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 20
RMSE_train= 817.9516437950333 RMSE_test= 2536.970975286381

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 50
RMSE_train= 256.1325867971387 RMSE_test= 2716.9373056097443

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 100
RMSE_train= 52.270681392992095 RMSE_test= 2794.6759796126435

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 200
RMSE_train= 3.665397389523359 RMSE_test= 2809.9002350688315

max_depth= None max_features= auto min_samples_leaf= 2 min_sample

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 2 n_estimators= 1400
RMSE_train= 0.0008951256921130681 RMSE_test= 2566.7189231932925

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 2 n_estimators= 1600
RMSE_train= 0.0003724680119505573 RMSE_test= 2566.718889963533

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 2 n_estimators= 1800
RMSE_train= 0.0003177093872885782 RMSE_test= 2566.718889269231

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 2 n_estimators= 2000
RMSE_train= 0.000315386656793031 RMSE_test= 2566.7188882427977

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 10
RMSE_train= 1676.667578284162 RMSE_test= 2233.517748629175

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 20
RMSE_train= 1318.7941953164825 RMSE_test= 2184.8338408013415

max_depth= None max_features= auto min_samples_leaf= 4 

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 800
RMSE_train= 0.0003151094920511765 RMSE_test= 2321.8798865048225

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 1000
RMSE_train= 0.0003151094920510755 RMSE_test= 2321.8798865048225

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 1200
RMSE_train= 0.0003151094920510755 RMSE_test= 2321.8798865048225

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 1400
RMSE_train= 0.0003151094920510755 RMSE_test= 2321.8798865048225

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 1600
RMSE_train= 0.0003151094920510755 RMSE_test= 2321.8798865048225

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 1800
RMSE_train= 0.0003151094920510755 RMSE_test= 2321.8798865048225

max_depth= None max_features= sqrt min_sa

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 10 n_estimators= 100
RMSE_train= 66.73023363329435 RMSE_test= 2290.0649561253717

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 10 n_estimators= 200
RMSE_train= 7.757367475969106 RMSE_test= 2312.849569631371

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 10 n_estimators= 400
RMSE_train= 0.20099885083549393 RMSE_test= 2314.7453200526834

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 10 n_estimators= 600
RMSE_train= 0.007341705362429567 RMSE_test= 2314.7802189634867

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 10 n_estimators= 800
RMSE_train= 0.0004362423018933658 RMSE_test= 2314.78133711866

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1000
RMSE_train= 0.00031420269937456333 RMSE_test= 2314.781415650432

max_depth= None max_features= sqrt min_samples_leaf= 

In [21]:
print("min RMSE in testset=",np.min(avg_RMSE1_test))
print("parameters:")
n1,n2,n3,n4,n5= np.unravel_index(np.argmin(avg_RMSE1_test), avg_RMSE1_test.shape)
print("max_depth=",param_grid["max_depth"][n1],"max_features=",param_grid["max_features"][n2],"min_samples_leaf=",param_grid["min_samples_leaf"][n3],"min_samples_split=",param_grid["min_samples_split"][n4],"n_estimators=",param_grid["n_estimators"][n5])
print("")

min RMSE in testset= 2099.8437116527166
parameters:
max_depth= 20 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 100



In [22]:
reg = GradientBoostingRegressor(n_estimators=param_grid["n_estimators"][n5],max_depth=param_grid["max_depth"][n1],min_samples_leaf=param_grid["min_samples_leaf"][n3],min_samples_split=param_grid["min_samples_split"][n4], max_features=param_grid["max_features"][n2],random_state=42)
reg.fit(df,df_y)
pred = reg.predict(df)
print("MSE=",mean_squared_error(df_y, pred))

MSE= 76862.062732602


In [23]:

avg_RMSE2_train=np.zeros((8,2,3,3,14))
avg_RMSE2_test=np.zeros((8,2,3,3,14))
for n1,max_depth in enumerate(param_grid['max_depth']):
    for n2,max_features in enumerate(param_grid['max_features']):
        for n3,min_samples_leaf in enumerate(param_grid['min_samples_leaf']):
            for n4,min_samples_split in enumerate(param_grid['min_samples_split']):
                for n5,n_estimators in enumerate(param_grid['n_estimators']):
                    MSE_train=[]
                    MSE_test=[]
                    oob_error=[]
                    total_train=0
                    total_test=0
                    for train_index, test_index in kf.split(df_2):
                        X_train= df_2.iloc[train_index]
                        y_train= df_y_2.iloc[train_index]
                        X_test= df_2.iloc[test_index]
                        y_test= df_y_2.iloc[test_index]
                        reg = GradientBoostingRegressor(n_estimators=n_estimators,max_depth=max_depth,min_samples_leaf=min_samples_leaf,min_samples_split=min_samples_split, max_features=max_features,random_state=42)
                        reg.fit(X_train,y_train)
                        pred_train = reg.predict(X_train)
                        pred_test = reg.predict(X_test)
                        MSE_train.append(mean_squared_error(y_train, pred_train)*len(train_index))
                        MSE_test.append(mean_squared_error(y_test, pred_test)*len(test_index))
                        total_train=total_train+len(train_index)
                        total_test=total_test+len(test_index)

                    avg_RMSE2_test[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_test)/total_test)
                    avg_RMSE2_train[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_train)/total_train)
                    print("max_depth=",max_depth,"max_features=",max_features,"min_samples_leaf=",min_samples_leaf,"min_samples_split=",min_samples_split,"n_estimators=",n_estimators)
                    print("RMSE_train=",avg_RMSE2_train[n1,n2,n3,n4,n5],"RMSE_test=",avg_RMSE2_test[n1,n2,n3,n4,n5])
                    print("")

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 10
RMSE_train= 3721.185998518828 RMSE_test= 6082.877243941719

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 20
RMSE_train= 1297.568510117534 RMSE_test= 5982.26126940366

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 50
RMSE_train= 55.02274911930852 RMSE_test= 6335.885996283357

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 100
RMSE_train= 0.2838042834348264 RMSE_test= 6356.05206259691

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 200
RMSE_train= 0.00031536143479628036 RMSE_test= 6356.160204176903

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 400
RMSE_train= 0.0003153614350872762 RMSE_test= 6356.160204176903

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimat

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1600
RMSE_train= 0.00031499087876681416 RMSE_test= 5926.353630090672

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1800
RMSE_train= 0.00031499087876681416 RMSE_test= 5926.353630090672

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 2000
RMSE_train= 0.00031499087876681416 RMSE_test= 5926.353630090672

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 10
RMSE_train= 3997.8941512530746 RMSE_test= 6175.641688162399

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 20
RMSE_train= 1644.6817987583959 RMSE_test= 5740.623945133721

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 50
RMSE_train= 242.47158016907915 RMSE_test= 5980.244446788338

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split

max_depth= 10 max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1000
RMSE_train= 0.07346382830432871 RMSE_test= 5770.03483547773

max_depth= 10 max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1200
RMSE_train= 0.01235410118038543 RMSE_test= 5770.036882581877

max_depth= 10 max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1400
RMSE_train= 0.00207689520641754 RMSE_test= 5770.037360705714

max_depth= 10 max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1600
RMSE_train= 0.00047012250273842203 RMSE_test= 5770.037409668645

max_depth= 10 max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1800
RMSE_train= 0.0003258485661912386 RMSE_test= 5770.037450418619

max_depth= 10 max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 2000
RMSE_train= 0.00031596087660873927 RMSE_test= 5770.037460271122

max_depth= 10 max_features= auto min_samples_leaf= 4 min_sample

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 400
RMSE_train= 0.013692147170825775 RMSE_test= 5076.585438247327

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 600
RMSE_train= 0.0003141819389582715 RMSE_test= 5076.585830284527

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 800
RMSE_train= 0.00031418193898169163 RMSE_test= 5076.585830284527

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 1000
RMSE_train= 0.00031418193898705745 RMSE_test= 5076.585830284527

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 1200
RMSE_train= 0.00031418193898705745 RMSE_test= 5076.585830284527

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 1400
RMSE_train= 0.00031418193898705745 RMSE_test= 5076.585830284527

max_depth= 10 max_features= sqrt min_samples_leaf= 1

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 100
RMSE_train= 559.8640956677599 RMSE_test= 5609.490471555106

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 200
RMSE_train= 204.67588118183926 RMSE_test= 5697.457109569661

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 400
RMSE_train= 35.21827701324864 RMSE_test= 5737.519761737694

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 600
RMSE_train= 6.434539155838328 RMSE_test= 5743.112646023821

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 800
RMSE_train= 1.242626081174608 RMSE_test= 5743.514171397977

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 1000
RMSE_train= 0.2386511292660437 RMSE_test= 5743.611868504275

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimator

...

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 5 n_estimators= 50
RMSE_train= 90.580151699101 RMSE_test= 5320.671059783642

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 5 n_estimators= 100
RMSE_train= 1.4247977258767128 RMSE_test= 5325.456379893672

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 5 n_estimators= 200
RMSE_train= 0.0006492762732687962 RMSE_test= 5325.422586415213

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 5 n_estimators= 400
RMSE_train= 0.00031366887822325927 RMSE_test= 5325.42251388191

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 5 n_estimators= 600
RMSE_train= 0.00031366887822304075 RMSE_test= 5325.42251388191

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 5 n_estimators= 800
RMSE_train= 0.00031366887822304075 RMSE_test= 5325.42251388191

max_depth= None max_features= auto min_samples_leaf= 1 min_sam

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 2000
RMSE_train= 0.0003143097748340081 RMSE_test= 5959.158566353856

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 10
RMSE_train= 4363.516577448678 RMSE_test= 5959.533765292746

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 20
RMSE_train= 2085.6011733694413 RMSE_test= 5185.205881422325

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 50
RMSE_train= 526.0813567255486 RMSE_test= 5499.684051861685

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 100
RMSE_train= 95.25599762172979 RMSE_test= 5602.935864846885

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 200
RMSE_train= 4.06770332766503 RMSE_test= 5609.3194122984305

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_s

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 10 n_estimators= 1400
RMSE_train= 0.0039027131317282138 RMSE_test= 5695.922678702842

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 10 n_estimators= 1600
RMSE_train= 0.0007817915905590295 RMSE_test= 5695.922769629823

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 10 n_estimators= 1800
RMSE_train= 0.0003482418690838351 RMSE_test= 5695.9228355187115

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 10 n_estimators= 2000
RMSE_train= 0.00031522757572667636 RMSE_test= 5695.9228513800745

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 2 n_estimators= 10
RMSE_train= 3720.898965243307 RMSE_test= 5869.5117559850205

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 2 n_estimators= 20
RMSE_train= 1297.3972469707405 RMSE_test= 5138.9925970633885

max_depth= None max_features= sqrt min_samples_l

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 2 n_estimators= 600
RMSE_train= 0.0003149012201810783 RMSE_test= 5225.80854176114

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 2 n_estimators= 800
RMSE_train= 0.0003149012201417376 RMSE_test= 5225.80854176114

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1000
RMSE_train= 0.0003149012201271455 RMSE_test= 5225.80854176114

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1200
RMSE_train= 0.00031490122012605104 RMSE_test= 5225.80854176114

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1400
RMSE_train= 0.00031490122012605104 RMSE_test= 5225.80854176114

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1600
RMSE_train= 0.00031490122012605104 RMSE_test= 5225.80854176114

max_depth= None max_features= sqrt min_samples_leaf

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 100
RMSE_train= 532.5438383282396 RMSE_test= 5540.228443864717

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 200
RMSE_train= 193.53176636819694 RMSE_test= 5623.03108918207

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 400
RMSE_train= 32.7380809294189 RMSE_test= 5653.72456101521

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 600
RMSE_train= 5.804798314313749 RMSE_test= 5658.885649669709

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 800
RMSE_train= 1.1561019944573596 RMSE_test= 5659.558235696747

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1000
RMSE_train= 0.2387685348965213 RMSE_test= 5659.680678646294

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5

In [24]:
print("min RMSE in testset=",np.min(avg_RMSE2_test))
print("parameters:")
n1,n2,n3,n4,n5= np.unravel_index(np.argmin(avg_RMSE2_test), avg_RMSE2_test.shape)
print("max_depth=",param_grid["max_depth"][n1],"max_features=",param_grid["max_features"][n2],"min_samples_leaf=",param_grid["min_samples_leaf"][n3],"min_samples_split=",param_grid["min_samples_split"][n4],"n_estimators=",param_grid["n_estimators"][n5])
print("")

min RMSE in testset= 4880.6072998766585
parameters:
max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 50



In [25]:
reg = GradientBoostingRegressor(n_estimators=param_grid["n_estimators"][n5],max_depth=param_grid["max_depth"][n1],min_samples_leaf=param_grid["min_samples_leaf"][n3],min_samples_split=param_grid["min_samples_split"][n4], max_features=param_grid["max_features"][n2],random_state=42)
reg.fit(df_2,df_y_2)
pred = reg.predict(df_2)
print("MSE=",mean_squared_error(df_y_2, pred))

MSE= 11799.897141442352


In [26]:

avg_RMSE3_train=np.zeros((8,2,3,3,14))
avg_RMSE3_test=np.zeros((8,2,3,3,14))
for n1,max_depth in enumerate(param_grid['max_depth']):
    for n2,max_features in enumerate(param_grid['max_features']):
        for n3,min_samples_leaf in enumerate(param_grid['min_samples_leaf']):
            for n4,min_samples_split in enumerate(param_grid['min_samples_split']):
                for n5,n_estimators in enumerate(param_grid['n_estimators']):
                    MSE_train=[]
                    MSE_test=[]
                    oob_error=[]
                    total_train=0
                    total_test=0
                    for train_index, test_index in kf.split(df_3):
                        X_train= df_3.iloc[train_index]
                        y_train= df_y_3.iloc[train_index]
                        X_test= df_3.iloc[test_index]
                        y_test= df_y_3.iloc[test_index]
                        reg = GradientBoostingRegressor(n_estimators=n_estimators,max_depth=max_depth,min_samples_leaf=min_samples_leaf,min_samples_split=min_samples_split, max_features=max_features,random_state=42)
                        reg.fit(X_train,y_train)
                        pred_train = reg.predict(X_train)
                        pred_test = reg.predict(X_test)
                        MSE_train.append(mean_squared_error(y_train, pred_train)*len(train_index))
                        MSE_test.append(mean_squared_error(y_test, pred_test)*len(test_index))
                        total_train=total_train+len(train_index)
                        total_test=total_test+len(test_index)

                    avg_RMSE3_test[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_test)/total_test)
                    avg_RMSE3_train[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_train)/total_train)
                    print("max_depth=",max_depth,"max_features=",max_features,"min_samples_leaf=",min_samples_leaf,"min_samples_split=",min_samples_split,"n_estimators=",n_estimators)
                    print("RMSE_train=",avg_RMSE3_train[n1,n2,n3,n4,n5],"RMSE_test=",avg_RMSE3_test[n1,n2,n3,n4,n5])
                    print("")

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 10
RMSE_train= 624.565382887185 RMSE_test= 879.9373502659647

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 20
RMSE_train= 217.79167468428264 RMSE_test= 740.6254525437733

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 50
RMSE_train= 9.23439634271826 RMSE_test= 749.8504777529994

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 100
RMSE_train= 0.0476027050165114 RMSE_test= 751.4323992214862

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 200
RMSE_train= 0.00031361915064663824 RMSE_test= 751.4412801773764

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 400
RMSE_train= 0.00031361915065696087 RMSE_test= 751.4412801773764

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estim

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1600
RMSE_train= 0.00031470553931170524 RMSE_test= 760.4917678814284

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1800
RMSE_train= 0.00031470553931170524 RMSE_test= 760.4917678814284

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 2000
RMSE_train= 0.00031470553931170524 RMSE_test= 760.4917678814284

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 10
RMSE_train= 659.0570921330071 RMSE_test= 867.7072683799225

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 20
RMSE_train= 262.4424293977985 RMSE_test= 739.4020689458192

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 50
RMSE_train= 49.5544640172258 RMSE_test= 743.1012249719657

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 5 

max_depth= 10 max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1000
RMSE_train= 0.012969795988418395 RMSE_test= 738.7878614395635

max_depth= 10 max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1200
RMSE_train= 0.0037693537310331646 RMSE_test= 738.7867355775252

max_depth= 10 max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1400
RMSE_train= 0.001285002536645928 RMSE_test= 738.7865234398646

max_depth= 10 max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1600
RMSE_train= 0.0005486971698117642 RMSE_test= 738.7865576884553

max_depth= 10 max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1800
RMSE_train= 0.00036665917460803715 RMSE_test= 738.786566065462

max_depth= 10 max_features= auto min_samples_leaf= 4 min_samples_split= 5 n_estimators= 2000
RMSE_train= 0.000326361423124509 RMSE_test= 738.7865685622721

max_depth= 10 max_features= auto min_samples_leaf= 4 min_samp

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 400
RMSE_train= 0.003249273410551759 RMSE_test= 682.096680323056

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 600
RMSE_train= 0.00031407425671050167 RMSE_test= 682.0968863946495

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 800
RMSE_train= 0.00031407425671190603 RMSE_test= 682.0968863946495

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 1000
RMSE_train= 0.0003140742567118738 RMSE_test= 682.0968863946495

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 1200
RMSE_train= 0.00031407425671177463 RMSE_test= 682.0968863946495

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 1400
RMSE_train= 0.00031407425671176254 RMSE_test= 682.0968863946495

max_depth= 10 max_features= sqrt min_samples_leaf= 1 

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 50
RMSE_train= 168.04009837954035 RMSE_test= 625.3069904360964

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 100
RMSE_train= 76.41723226757351 RMSE_test= 639.2838841867826

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 200
RMSE_train= 23.29444838050263 RMSE_test= 646.4012859143799

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 400
RMSE_train= 3.1899864984457644 RMSE_test= 649.7351398577749

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 600
RMSE_train= 0.4855897012897358 RMSE_test= 649.5588041905662

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 800
RMSE_train= 0.07554574582104547 RMSE_test= 649.566971815056

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimator

...

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 800
RMSE_train= 0.00031317411036444175 RMSE_test= 735.031208876749

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1000
RMSE_train= 0.0003131741103648914 RMSE_test= 735.031208876749

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1200
RMSE_train= 0.0003131741103648914 RMSE_test= 735.031208876749

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1400
RMSE_train= 0.0003131741103648914 RMSE_test= 735.031208876749

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1600
RMSE_train= 0.0003131741103648914 RMSE_test= 735.031208876749

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1800
RMSE_train= 0.0003131741103648914 RMSE_test= 735.031208876749

max_depth= None max_features= auto min_samples_leaf=

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 200
RMSE_train= 0.9292519934882841 RMSE_test= 752.271696116135

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 400
RMSE_train= 0.007687930511798485 RMSE_test= 752.1903195330856

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 600
RMSE_train= 0.0003219147683223497 RMSE_test= 752.1898377175607

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 800
RMSE_train= 0.0003147416481772736 RMSE_test= 752.1898426312962

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1000
RMSE_train= 0.0003147416481771928 RMSE_test= 752.1898426312962

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1200
RMSE_train= 0.0003147416481783167 RMSE_test= 752.1898426312962

max_depth= None max_features= auto min_samples_leaf= 2 

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 10 n_estimators= 100
RMSE_train= 81.75432473403198 RMSE_test= 701.0081606689755

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 10 n_estimators= 200
RMSE_train= 25.65820450416992 RMSE_test= 718.026041661851

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 10 n_estimators= 400
RMSE_train= 3.3755727632232655 RMSE_test= 727.2382909961793

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 10 n_estimators= 600
RMSE_train= 0.5041115045866023 RMSE_test= 728.4913770136849

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 10 n_estimators= 800
RMSE_train= 0.07832919846930525 RMSE_test= 728.6880615691312

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 10 n_estimators= 1000
RMSE_train= 0.013994473855672786 RMSE_test= 728.7164933994414

max_depth= None max_features= auto min_samples_leaf= 4 min_samp

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 2000
RMSE_train= 0.00031281830419822946 RMSE_test= 696.7486922631599

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 2 n_estimators= 10
RMSE_train= 663.2319944933724 RMSE_test= 863.1472242517738

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 2 n_estimators= 20
RMSE_train= 266.0058824391709 RMSE_test= 698.1602988076

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 2 n_estimators= 50
RMSE_train= 43.9524464175548 RMSE_test= 692.1941474281335

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 2 n_estimators= 100
RMSE_train= 8.758421910258175 RMSE_test= 698.3552782894217

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 2 n_estimators= 200
RMSE_train= 0.5123132519131431 RMSE_test= 699.2614387926817

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 2

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 1400
RMSE_train= 0.0008128197097254095 RMSE_test= 649.9173525451941

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 1600
RMSE_train= 0.0004365361916489269 RMSE_test= 649.9173550717869

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 1800
RMSE_train= 0.00033802776444622965 RMSE_test= 649.9173743849357

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 2000
RMSE_train= 0.0003179404540424087 RMSE_test= 649.9173739726809

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 10
RMSE_train= 720.7644945213411 RMSE_test= 830.913658827225

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 20
RMSE_train= 365.07347843898714 RMSE_test= 629.4734122669314

max_depth= None max_features= sqrt min_samples_leaf= 4 mi

In [27]:
print("min RMSE in testset=",np.min(avg_RMSE3_test))
print("parameters:")
n1,n2,n3,n4,n5= np.unravel_index(np.argmin(avg_RMSE3_test), avg_RMSE3_test.shape)
print("max_depth=",param_grid["max_depth"][n1],"max_features=",param_grid["max_features"][n2],"min_samples_leaf=",param_grid["min_samples_leaf"][n3],"min_samples_split=",param_grid["min_samples_split"][n4],"n_estimators=",param_grid["n_estimators"][n5])
print("")

min RMSE in testset= 625.3069904360964
parameters:
max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 50



In [28]:
reg = GradientBoostingRegressor(n_estimators=param_grid["n_estimators"][n5],max_depth=param_grid["max_depth"][n1],min_samples_leaf=param_grid["min_samples_leaf"][n3],min_samples_split=param_grid["min_samples_split"][n4], max_features=param_grid["max_features"][n2],random_state=42)
reg.fit(df_3,df_y_3)
pred = reg.predict(df_3)
print("MSE=",mean_squared_error(df_y_3, pred))

MSE= 24664.931425435596
